In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import re
from langdetect import detect
from sklearn.model_selection import train_test_split

2025-04-24 10:50:46.778115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745484647.417693   24040 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745484647.591635   24040 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745484649.138689   24040 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745484649.138716   24040 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745484649.138717   24040 computation_placer.cc:177] computation placer alr

In [5]:
#load the dataset
print("loading airbnb reviews dataset...")
df = pd.read_csv("reviews.csv")
print(f"Orginal dataset\n {df}")

loading airbnb reviews dataset...
Orginal dataset
                  listing_id                   id        date  reviewer_id  \
0                    164448               407660  2011-07-30       870312   
1                    164448               451097  2011-08-16       901633   
2                    164448               472271  2011-08-24       894674   
3                    164448               521708  2011-09-11       232485   
4                    164448               568347  2011-09-26       896712   
...                     ...                  ...         ...          ...   
144943  1305729897949100039  1317299844322463966  2024-12-22    666963478   
144944  1306393577432561110  1315873156254706149  2024-12-20    442177472   
144945  1310885179474906446  1316587888366107935  2024-12-21     53058534   
144946  1312171107579620356  1320280905319488958  2024-12-26    595482193   
144947  1316786620019656619  1322401494741586698  2024-12-29    135251598   

       reviewer_name    

In [6]:
# Clean html tags
def clean_html(text):
    if isinstance(text, str):
        return re.sub(r"<.*?>", "", text)
    return text

In [7]:
# Remove the empty comments rows
df = df.dropna(subset=["comments"])
df["comments"] = df["comments"].apply(clean_html)
print(f"After remoing empty comments\n {df}")

After remoing empty comments
                  listing_id                   id        date  reviewer_id  \
0                    164448               407660  2011-07-30       870312   
1                    164448               451097  2011-08-16       901633   
2                    164448               472271  2011-08-24       894674   
3                    164448               521708  2011-09-11       232485   
4                    164448               568347  2011-09-26       896712   
...                     ...                  ...         ...          ...   
144943  1305729897949100039  1317299844322463966  2024-12-22    666963478   
144944  1306393577432561110  1315873156254706149  2024-12-20    442177472   
144945  1310885179474906446  1316587888366107935  2024-12-21     53058534   
144946  1312171107579620356  1320280905319488958  2024-12-26    595482193   
144947  1316786620019656619  1322401494741586698  2024-12-29    135251598   

       reviewer_name                         

In [ ]:
# Filter out the rows that isn't in English
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False
    
df["is_english"] = df["comments"].apply(is_english)
df = df[df["is_english"]]
print(f"After removing non-english comments\n {df}")

In [ ]:
print("Loading pre-trained BERT model for sentiment analysis...")
# load a pre-trained bert model from tensorflow hub
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

Loading pre-trained BERT model for sentiment analysis...


In [ ]:
# build the model
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # forces CPU-only


def build_classifier_model():
    with tf.device("/CPU:0"):
        hub_layer = hub.KerasLayer(model_url, trainable=False)

    model = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    
    return model


In [ ]:
# create the model
print("Creating labels for training model...")
positive_words = ['great', 'good', 'nice', 'excellent', 'perfect', 'happy', 'wonderful', 
                 'fantastic', 'amazing', 'love', 'best', 'beautiful', 'clean', 'comfortable']
negative_words = ['bad', 'poor', 'terrible', 'horrible', 'awful', 'worst', 'dirty', 
                 'disappointing', 'disappointment', 'uncomfortable', 'problem', 'not clean']

Creating labels for training model...


In [ ]:
def simple_sentiment(text):
    if not isinstance(text, str):
        return 1
    
    text = text.lower()
    pos_count = sum(1 for word in positive_words if word in text)
    neg_count = sum(1 for word in negative_words if word in text)

    if neg_count > pos_count:
        return 0 # negative
    else:
        return 1 # Default to positive

In [ ]:
# create simple labels
df["sentiment_label"] = df["comments"].apply(simple_sentiment)

In [ ]:
# sample subset for training
sample_size = min(10000, len(df))
sample_df = df.sample(sample_size, random_state=42)

In [ ]:
# split into training and validation sets
train_text, val_text, train_labels, val_labels = train_test_split(
    sample_df["comments"].values,
    sample_df["sentiment_label"].values,
    test_size=0.2,
    random_state=42
)

In [ ]:
# create and train the model
model = build_classifier_model()
history = model.fit(
    x=train_text,
    y=train_labels,
    validation_data=(val_text, val_labels),
    epochs=5,
    batch_size=8,
)

NameError: name 'tf' is not defined